<a href="https://colab.research.google.com/github/Sorrellee/Data-mining-technologies/blob/main/%D0%A2%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Разработка модели предсказания реакции на маркетинговые кампании на основе анализа социальных данных и поведения потребителей

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

In [96]:
# Загрузка данных
df = pd.read_csv('/content/Marketingcampaigns.csv')

In [97]:
df

,Customer id,Age,Gender,Location,Email Opened,Email Clicked,Product page visit,Discount offered,Purchased
0,1,22,0,Perth,1,1,3,1,1
1,2,55,0,Auckland,1,0,0,0,0
2,3,15,1,Sydney,0,1,2,1,1
3,4,25,0,Brisbane,1,1,5,1,0
4,5,36,1,Brisbane,0,1,1,1,0
5,6,30,0,Sydney,0,0,0,0,0
6,7,28,1,Sydney,0,0,3,1,1
7,8,19,1,Sydney,1,1,2,0,0
8,9,59,0,Perth,1,1,1,0,0
9,10,45,1,Auckland,0,0,0,0,0


In [98]:
# Определение признаков и целевой переменной
features = ['Email Opened', 'Email Clicked', 'Discount offered']
target_variable = 'Purchased'
X = df[features]
y = df[target_variable]

In [99]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
# Стандартизация признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [101]:
# Подбор параметров с помощью Grid Search для RandomForestClassifier
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [None, 10, 20, 30],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

In [102]:
# Кросс-валидированная точность
scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
print(f'Кросс-валидированная точность: {scores.mean()}')

Кросс-валидированная точность: 0.41666666666666663


In [103]:
# Точность на тестовой выборке
print(f'Точность на тестовой выборке: {best_model.score(X_test, y_test)}')

Точность на тестовой выборке: 0.75


In [104]:
# Важность признаков
feature_importances = best_model.feature_importances_
print('\nВажность признаков:')
for feature, importance in zip(features, feature_importances):
    print(f'{feature}: {importance:.4f}')


Важность признаков:
Email Opened: 0.2752
Email Clicked: 0.3257
Discount offered: 0.3991


In [105]:
# Создание DataFrame с фактическими и предсказанными метками
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': best_model.predict(X_test)})

# Сохранение DataFrame в CSV-файл
results_df.to_csv('/content/model_results.csv', index=False)